In [26]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

Ideas:

- what do agencies spend the money on? Let's use the same ideas we have been working with. 
- clustering algorithm for all agencies and for all the years. what is the evolution? obamacare?



## the first question: do different counties spend in different things?

## load data and preprocess transaction_description

In [36]:
DATAFILE_model = '/Users/rodrigo/Documents/Big-Data/final-project/hhs_ca/ML/df_model.csv'

df = pd.read_csv(DATAFILE_model, low_memory = False)
df = df[['total_obligation', 'transaction_description', 'business_categories', 'recipient_location_county_code']]
df = df[pd.notnull(df['transaction_description'])]

#t = df[df.transaction_description  != ""]
#df = df[pd.notnull(df['total_obligation'])]

print(df.shape)
df.head()

(207892, 4)


,total_obligation,transaction_description,business_categories,recipient_location_county_code
0,4468.0,TAS::75 0943::TAS 00HCCMBG-2009-70054 - MINIGU...,small_business limited_liability_corporation c...,113
1,574970.0,ACCESSIBLE ENVIRONMENTAL INFORMATION APPLICATI...,small_business category_business,113
2,75000.0,ACCESSIBLE ENVIRONMENTAL INFORMATION APPLICATI...,small_business category_business,113
3,574970.0,ACCESSIBLE ENVIRONMENTAL INFORMATION APPLICATI...,small_business category_business,113
4,574970.0,ACCESSIBLE ENVIRONMENTAL INFORMATION APPLICATI...,small_business category_business,113


In [31]:
tr_descriptions = df['transaction_description']

snowball = nltk.stem.SnowballStemmer('english')

temp = []

for transaction in tr_descriptions:
    transaction = transaction.lower()                                            # changing to lower case
    transaction = "".join(l for l in transaction if l not in string.punctuation) # removing puntuation
    transaction = ''.join([l for l in transaction if not l.isdigit()])           # removing numbers
    words = [snowball.stem(word) for word in transaction.split()                 # stemming the words
             if word not in stopwords.words('english')]                          # remvoving stopwords
    temp.append(words)

# flatten the list
sent = []
for row in temp:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)
    
tr_descriptions = sent

In [38]:
df['transaction_description'] = pd.Series(tr_descriptions)
df.head()

,total_obligation,transaction_description,business_categories,recipient_location_county_code,transaction_descriptioni
0,4468.0,tas tas hccmbg miniguid ultrason devic trevit,small_business limited_liability_corporation c...,113,tas tas hccmbg miniguid ultrason devic trevit
1,574970.0,access environment inform applic individu vis...,small_business category_business,113,access environment inform applic individu vis...
2,75000.0,access environment inform applic individu vis...,small_business category_business,113,access environment inform applic individu vis...
3,574970.0,access environment inform applic individu vis...,small_business category_business,113,access environment inform applic individu vis...
4,574970.0,access environment inform applic individu vis...,small_business category_business,113,access environment inform applic individu vis...


## divide by county

In [139]:
list_counties = list(df.groupby('recipient_location_county_code'))

# weighthing schemes

## tfidf

In [288]:
# reset index?

def tfidf_words_weights_sorted(tuple_):
    df = tuple_[1]
    df = df[pd.notnull(df['transaction_description'])]
    tr_descriptions = df['transaction_description']
    
    #weighting scheme
    
    try:
        tfidf_transformer = TfidfVectorizer()
        description_tfidf = tfidf_transformer.fit_transform(tr_descriptions)
    
    except ValueError:
        print('The county we need to check :' + str(tuple_[0]))
        return None
    
    # we need to extract the weights of the words
    index_value={i[1]:i[0] for i in tfidf_transformer.vocabulary_.items()}

    words_weights = []
    for row in description_tfidf:
        words_weights.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})
    
    # change dictionary to dataframe (easier to manipulate)
    temp = []
    for description in words_weights:
        w = pd.Series(description).to_frame(tuple_[0])
        temp.append(w)
    
    words_df = pd.concat(temp)
    
    #sum the weights of same words and sort them
    words_df_aggregate_sorted = words_df.groupby(words_df.index).sum().sort_values(
                                                                            by = [tuple_[0]],
                                                                            ascending = False)
    return words_df_aggregate_sorted  

In [289]:
x = list(map(tfidf_words_weights_sorted, list_counties))

The county we need to check :51


## amount

In [283]:
def list_to_df(list_):
    try:
        
        length = len(str(list_[1]).split())
        dollars = list_[0]/length
        df = pd.DataFrame({ 'word':str(list_[1]).split(),
                            'money': np.repeat(dollars, length)})
    except ZeroDivisionError:
        print('h')
        return None
    
    return df

def amount_words_weights_sorted(tuple_):
    df = tuple_[1]
    df = df[['total_obligation', 'transaction_description']]
    df = df[pd.notnull(df[['transaction_description', 'total_obligation']])]

    # create new df with a list with both values
    list_comb = df.values.tolist()
    temp = list(map(list_to_df, list_comb))
    words_money = pd.concat(temp)
    
    # we need to aggregate same words and sum them
    words_money_agregate_sorted = words_money.groupby('word').sum().sort_values(by = 'money', ascending = False)
    words_money_agregate_sorted.columns = [tuple_[0]]
    
    return words_money_agregate_sorted


In [ ]:
y = list(map(amount_words_weights_sorted, list_counties))`m

In [296]:
x[1]

,3
tas,0.666667
clear,0.500000
communic,0.500000
cultur,0.500000
hcpbcb,0.500000
addit,0.333333
ciosp,0.333333
contract,0.333333
er,0.333333
fund,0.333333


In [297]:
y[1]

,3
word,
clear,625000.0
communic,625000.0
cultur,625000.0
hcpbcb,625000.0
tas,625000.0
addit,312500.0
c,312500.0
ciosp,312500.0
contract,312500.0
